In [1]:
import sys, os
import cv2
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import pytesseract
from PIL import Image
from Levenshtein import distance
from pytesseract import Output
import imutils
import glob
import wget

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [2]:
VRNdf = pd.read_excel("VRNdf.xlsx")
VRNdf = VRNdf[:1821]
VRNdf

,Unnamed: 0,VRN,0
0,0,BHA6172,豫B
1,1,BMU2999,BW399
2,2,PFF9889,W6F9889
3,3,PTA2105,皖APTA21
4,4,CCD7351,皖ACC07
5,5,CDI4388,豫00T4388
6,6,PCT5326,6C53261
7,7,728JFW,宁28JF
8,8,PMZ8952,PMZ8952
9,9,AGV6998,AL6988


In [3]:
VRNdf['Distance'] = 0
VRNdf['Accuracy'] = 0
for i in range(VRNdf.shape[0]):
    VRNdf.iloc[i,3] = distance(VRNdf.iloc[i,1], VRNdf.iloc[i,2])
    VRNdf.iloc[i,4] = str(round(((len(VRNdf.iloc[i,1]) - VRNdf.iloc[i,3]) / len(VRNdf.iloc[i,1]))*100,1)) + '%'

In [5]:
VRNdf

,Unnamed: 0,VRN,0,Distance,Accuracy
0,0,BHA6172,豫B,7,0.0%
1,1,BMU2999,BW399,4,42.9%
2,2,PFF9889,W6F9889,2,71.4%
3,3,PTA2105,皖APTA21,4,42.9%
4,4,CCD7351,皖ACC07,6,14.3%
5,5,CDI4388,豫00T4388,4,42.9%
6,6,PCT5326,6C53261,3,57.1%
7,7,728JFW,宁28JF,2,66.7%
8,8,PMZ8952,PMZ8952,0,100.0%
9,9,AGV6998,AL6988,3,57.1%


In [6]:
listed = list(VRNdf['Accuracy'])
findacc = []
for i in listed:
    a = float(i.replace('%',''))
    findacc.append(a)
def Average(lst): 
    return sum(lst) / len(lst) 
average = Average(findacc) 
average


42.64887424492038

In [9]:
VRNdf.groupby('Distance').count()

(21/1821)*100

1.1532125205930808

In [28]:
VRNdf.to_excel("VRNdf_new.xlsx")